In [1]:
# 57.2 waala model hai
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Assuming df_train and 'winner' column exist in your dataset

# Select numerical columns
numerical_cols = df_train.select_dtypes(include=['float64', 'int64']).columns

# Exclude the target column from numerical_cols if needed
if 'winner' in numerical_cols:
    numerical_cols = numerical_cols.drop('winner')

# Exclude additional columns if necessary
numerical_cols = numerical_cols.drop(['team1_id', 'match id', 'team2_id', 'ground_id'])

# Split the data into train, test, and validation sets with stratification
X = df_train[top_features_cumulative]
y = df_train['winner']

# Split the data with stratification
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Initialize the models with default parameters
catboost_model = CatBoostClassifier(verbose=False, learning_rate=0.1)
xgb_model = XGBClassifier(verbosity=0, learning_rate=0.1)
lgbm_model = LGBMClassifier(learning_rate=0.1)
gbm_model = GradientBoostingClassifier(learning_rate=0.1)

# Train the models on the training set
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
lgbm_model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
gbm_model.fit(X_train, y_train)

# Get predictions for the validation set
catboost_val_pred = catboost_model.predict_proba(X_val)[:, 1]
xgb_val_pred = xgb_model.predict_proba(X_val)[:, 1]
lgbm_val_pred = lgbm_model.predict_proba(X_val)[:, 1]
gbm_val_pred = gbm_model.predict_proba(X_val)[:, 1]

# Stack predictions as new features for validation set
val_preds = np.column_stack((catboost_val_pred, xgb_val_pred, lgbm_val_pred, gbm_val_pred))

# Train the meta-model on the stacked predictions
meta_model = CatBoostClassifier(verbose=False, learning_rate=0.1)
meta_model.fit(val_preds, y_val)

# Perform cross-validation on the validation set and report min, max, and average scores
cv_scores = cross_val_score(meta_model, val_preds, y_val, cv=5, scoring='accuracy')
print(f"CV Scores - Avg: {cv_scores.mean()}, Min: {cv_scores.min()}, Max: {cv_scores.max()}")

# Get predictions for the test set
catboost_test_pred = catboost_model.predict_proba(X_test)[:, 1]
xgb_test_pred = xgb_model.predict_proba(X_test)[:, 1]
lgbm_test_pred = lgbm_model.predict_proba(X_test)[:, 1]
gbm_test_pred = gbm_model.predict_proba(X_test)[:, 1]

# Stack predictions as new features for test set
test_preds = np.column_stack((catboost_test_pred, xgb_test_pred, lgbm_test_pred, gbm_test_pred))

# Predict on the test set using the meta-model
y_pred_test = meta_model.predict(test_preds)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {test_accuracy}")

# Get feature importances
catboost_importances = catboost_model.get_feature_importance()
xgb_importances = xgb_model.feature_importances_
lgbm_importances = lgbm_model.feature_importances_
gbm_importances = gbm_model.feature_importances_

# Combine feature importances into a DataFrame
ft_imp_cat = pd.DataFrame({'Feature': X.columns, 'Importance': catboost_importances})
ft_imp_xgb = pd.DataFrame({'Feature': X.columns, 'Importance': xgb_importances})
ft_imp_lgbm = pd.DataFrame({'Feature': X.columns, 'Importance': lgbm_importances})
ft_imp_gbm = pd.DataFrame({'Feature': X.columns, 'Importance': gbm_importances})

NameError: name 'df_train' is not defined

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

# Load the data
file_path = "/content/processed_train_f (2).csv"
data = pd.read_csv(file_path)

# Dropping irrelevant columns
irrelevant_columns = [
    'match id', 'team1', 'team1_id', 'team1_roster_ids',
    'team2', 'team2_id', 'team2_roster_ids', 'venue', 'city',
    'match_dt', 'series_name', 'season'
]
data_cleaned = data.drop(columns=irrelevant_columns)

# Select only numeric columns
data_numeric = data_cleaned.select_dtypes(include=['number'])

# Fill missing values with the mean of their respective columns
data_filled = data_numeric.fillna(data_numeric.mean())

# Split the data into features and target variable
X = data_filled.drop(columns=['winner'])
y = data_filled['winner']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 100)
        self.fc2 = nn.Linear(100, 35)
        self.fc3 = nn.Linear(35, 2)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        embeddings = x  # Extract embeddings here
        x = self.fc3(x)
        return x, embeddings

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs, _ = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Extract embeddings from the trained model
model.eval()
_, embeddings = model(X_tensor)

embeddings = embeddings.detach().numpy()

print(f"Shape of embeddings: {embeddings.shape}")


from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# Train CatBoost model on the entire dataset embeddings
catboost_model = CatBoostClassifier(verbose=0)
catboost_model.fit(embeddings, y)

# Predict on the entire dataset
y_pred_catboost = catboost_model.predict(embeddings)

# Evaluate the CatBoost model
accuracy_catboost = accuracy_score(y, y_pred_catboost)
report_catboost = classification_report(y, y_pred_catboost)

# Perform cross-validation on the entire dataset
cv_scores = cross_val_score(catboost_model, embeddings, y, cv=5, scoring='accuracy', n_jobs=-1)

# Print the results
print(f"CatBoost Accuracy: {accuracy_catboost}")
print(f"CatBoost Classification Report:\n{report_catboost}")
print(f"Cross-Validation Scores - Avg: {cv_scores.mean()}, Min: {cv_scores.min()}, Max: {cv_scores.max()}")